In [1]:
import pandas as pd
df=pd.read_excel("/DS Internship - EDA - Data.xlsx")

In [2]:
df.head()

,Store,Year,Month,Super Division,Sales,SGM,Store Open,Store Close,Outlet Type,State,Climate,Total Sq Ft,Store Modification Date,Store Modification
0,1,2015,2,GIRLS,63635.80,35692.09,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
1,1,2015,2,BOYS,20872.93,15691.34,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
2,1,2015,2,ACCESSORIES,38717.03,23866.25,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
3,1,2015,2,KIDS,11455.00,12110.00,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
4,1,2015,3,GIRLS,67507.77,42237.79,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation


In [6]:
#1a
total_sales = df.groupby("Year")["Sales"].sum().reset_index()
total_sales


,Year,Sales
0,2015,1.627585e+09
1,2016,1.843939e+09
2,2017,1.651322e+09
3,2018,1.492340e+09
4,2019,1.471963e+09
5,2020,6.510899e+07


In [8]:
#1b
stores_opened1991= df[df["Store Open"].dt.year == 1991]["Store"].nunique()
print("stores opened in 1991 are:",stores_opened1991," stores")



stores opened in 1991 are: 4  stores


In [11]:
#1c
stores_remodeled=df["Store Modification"].notna().sum()
stores_remodeled

181237

In [12]:
#1d
salescorr=df["Sales"].corr(df["Total Sq Ft"])
salescorr


0.13922544019446748

In [14]:
#1e
profitable_division = df.groupby("Super Division")["Sales"].sum().reset_index().sort_values("Sales", ascending=False).head(1)
profitable_division

,Super Division,Sales
2,GIRLS,3.572426e+09


In [16]:
#1f
active_stores=df[df["Store Close"]=="No Close date"]["Store"].nunique()
active_stores

678

In [17]:
#1g
avg_sqft_by_division= df.groupby("Super Division")["Total Sq Ft"].mean().reset_index().sort_values("Total Sq Ft", ascending=False).head(1)
avg_sqft_by_division

,Super Division,Total Sq Ft
3,KIDS,6344.792952


In [19]:
#2a Advanced insights
df["Store Open"] = pd.to_datetime(df["Store Open"], errors='coerce')
df["Store Close"] = pd.to_datetime(df["Store Close"], errors='coerce')
df["Store Modification Date"] = pd.to_datetime(df["Store Modification Date"], errors='coerce')

In [22]:
state_performance = df.groupby("State").agg(
    total_sales=("Sales", "sum"),
    active_stores=("Store Close",lambda x: x.isna().sum()),
    avg_sq_ft=("Total Sq Ft", "mean")
).reset_index()
state_performance.head(5)

,State,total_sales,active_stores,avg_sq_ft
0,A1,3.738017e+07,710,5573.006236
1,A10,2.694839e+08,5172,6367.403152
2,A2,2.375814e+08,4300,6459.215333
3,A3,3.946254e+08,7208,6001.663180
4,A4,2.738552e+08,5496,6289.636738


In [23]:
state_performance["score"]=(state_performance["total_sales"]/1e9)+state_performance["active_stores"]+(state_performance["avg_sq_ft"]/1000)
top_3=state_performance.sort_values("score",ascending=False).head(3)


In [24]:

top_3

,State,total_sales,active_stores,avg_sq_ft,score
11,B10,866596553.3,16451,5920.746473,16457.787343
18,B8,472226869.2,9704,6521.288556,9710.993515
5,A5,458804278.1,8541,6764.964535,8548.223769


In [26]:
#2b
monthly_sales=df.groupby("Month")["Sales"].mean().reset_index().sort_values("Sales", ascending=False)
best_month=monthly_sales.head(1)
best_month

,Month,Sales
11,12,63392.905721


In [27]:
#2c
outlet_closure_rate=df.groupby("Outlet Type").agg(
    total_stores=("Store", "count"),
    closed_stores=("Store Close", lambda x:x.notna().sum())).reset_index()

In [29]:
outlet_closure_rate["closure_rate"] = (outlet_closure_rate["closed_stores"] / outlet_closure_rate["total_stores"]) * 100
outlet_closure_rate.sort_values("closure_rate",ascending=False)

,Outlet Type,total_stores,closed_stores,closure_rate
1,Outlet,21486,4845,22.549567
2,Strip,73518,13786,18.751870
0,Mall,86233,5122,5.939721
